In [14]:
import wandb
wandb.init(mode="disabled")

# Install and Import Packages

In [17]:
!pip install seqeval transformers accelerate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [16]:
import os
import numpy as np
import torch

from functools import partial
from seqeval.metrics import recall_score, precision_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

2024-06-02 18:50:20.989404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 18:50:20.989505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 18:50:21.118115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [18]:
from pii_dataloader import preprocess_data, get_dataset_from_path, get_train_val_test_split

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


# Evaluation

In [19]:
def get_fbeta_score(precision, recall, beta=5.0):
    b2 = beta ** 2
    return (1 + b2) * ((precision * recall) / (b2 * precision + recall))


def compute_metrics(p, labels_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    fbeta_score = get_fbeta_score(precision, recall)

    results = {
        'recall': recall,
        'precision': precision,
        'fbeta_score': fbeta_score
    }

    return results

# Configurations

In [21]:
# Model configuration

class CFG:
    LABELS_LIST = ['B-NAME_STUDENT', 'B-EMAIL', 'B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM', 'B-URL_PERSONAL', 'B-STREET_ADDRESS', 'I-NAME_STUDENT', 'I-EMAIL', 'I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM','I-URL_PERSONAL','I-STREET_ADDRESS', 'O']
    label2id = {label: i for i, label in enumerate(LABELS_LIST)}
    label2id['[PAD]'] = -100
    id2label = {i: label for label, i in label2id.items()}
    seed = 42
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = True

    # model checkpoint
    model_name = 'microsoft/deberta-base'
    train_head_only = False

    # path to data
    data_path = "/kaggle/input/pii-detection-removal-from-educational-data/train.json"

    # path to the directory where the model will be saved
    model_save_name = 'Doberta'
    target_dir = '/kaggle/working/model/trainer_'+model_save_name
    model_save_path = '/kaggle/working/model/model_'+model_save_name

    #training arguments
    training_args = TrainingArguments(
        output_dir=os.path.join(target_dir, 'trainer_args'),
        eval_strategy="epoch")

# Training

In [22]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained(CFG.model_name, num_labels=len(CFG.id2label), id2label=CFG.id2label, label2id=CFG.label2id)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
data = get_dataset_from_path(CFG.data_path)
data = preprocess_data(data, tokenizer, label2id = CFG.label2id)

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'],
    num_rows: 6807
})
encoding the labels...


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

tokenizing and aligning...


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

flattening the data...


100%|██████████| 5/5 [00:19<00:00,  3.86s/it]


In [24]:
data

Dataset({
    features: ['input_ids', 'attention_mask', 'org_word_ids', 'document', 'labels'],
    num_rows: 14123
})

In [26]:
data_train, data_eval, data_test = get_train_val_test_split(data, seed=CFG.seed)

In [27]:
trainer = Trainer(
    model=model,
    args=CFG.training_args,
    train_dataset=data_train,
    eval_dataset=data_eval,
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, labels_list=CFG.LABELS_LIST),
)

In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Recall,Precision,Fbeta Score
1,0.002900,0.000914,0.670886,0.898305,0.677483
2,0.000700,0.000721,0.943038,0.712919,0.931474
3,0.000400,0.000392,0.949367,0.909091,0.947752


TrainOutput(global_step=4290, training_loss=0.0033048921653301843, metrics={'train_runtime': 3224.9844, 'train_samples_per_second': 10.641, 'train_steps_per_second': 1.33, 'total_flos': 1.046038349832192e+16, 'train_loss': 0.0033048921653301843, 'epoch': 3.0})

In [29]:
trainer.evaluate(data_test, metric_key_prefix='test')

{'test_loss': 0.0008765372331254184,
 'test_recall': 0.8392857142857143,
 'test_precision': 0.875,
 'test_fbeta_score': 0.8406053459119496,
 'test_runtime': 47.6686,
 'test_samples_per_second': 29.642,
 'test_steps_per_second': 3.713,
 'epoch': 3.0}